# 常规赛：用户购买预测5月第9名
注：本项目参考于PGL系列：用户购买预测Baseline


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
import time
raw = pd.read_csv('data/data19383/train.csv')

# 将goods_id节点重新编号
raw['goods_id'] = pd.factorize(raw['goods_id'])[0]
# 将raw根据customer_id排序并将重新编号
submission_list = pd.DataFrame(raw.groupby('customer_id')['customer_gender'].last()).fillna(0)

raw = raw.sort_values(['customer_id'])
raw['customer_id'] = pd.factorize(raw['customer_id'])[0]

# 8月之前的数据作为训练集
train_raw = raw[raw['order_pay_time'] < '2013-07-31 23:59:59']

# 根据customer_id进行提取
train_data = pd.DataFrame(train_raw.groupby('customer_id')['customer_gender'].last().fillna(0))
# 在数据集每出现一次作为一个订单，统计每一个用户出现的次数，作为购买的频次
train_data['order_detail_count'] = train_raw.groupby('customer_id')['customer_id'].count()

# 8月份的数据作为label_raw
label_raw = set(raw[raw['order_pay_time'] > '2013-07-31 23:59:59']['customer_id'].dropna())
# 如果该用户在8月份完成了购买 label=1, 否则为0
train_data['labels'] = train_data.index.map(lambda x: int(x in label_raw))

# 最后test阶段要对所有的节点做预测，使用全部的raw作为测试数据
test_data = pd.DataFrame(raw.groupby('customer_id')['customer_gender'].last().fillna(0))
test_data['order_detail_count'] = raw.groupby('customer_id')['customer_id'].count()

我们对之前的猜想1进行简单验证。对订单数进行排序，

In [3]:
# 将train_data按照订单数从小到大排序
td = train_data.sort_values(['order_detail_count']).reset_index()
from matplotlib import pyplot as plt
plt.figure(figsize=(10,4))
plt.xlim(0, 1600000)
plt.plot(td['order_detail_count'], 'r.')
plt.show()
# 取td中下个月购买的数据，其index即在订单数中的排序
idx = td[td['labels'] == 1].index
# print(idx)
import seaborn as sns
sns.boxplot(idx)

<Figure size 1000x400 with 1 Axes>

In [5]:
# 用户的节点数量
customer_id = test_data.index
print(len(customer_id))

1585986


In [6]:
# 提取所要用到的raw数据
customer_id = test_data.index
customer_raw = test_data['order_detail_count']

edge_raw = raw[['customer_id', 'goods_id']]

goods_id = np.sort(edge_raw['goods_id'].unique())
goods_raw = raw.groupby('goods_id')['goods_id'].count()

# 所有节点数，即用户和商品的总和
num_custome = len(customer_id)
num_goods = len(goods_id)
num_nodes = num_goods + num_custome

# 为每一个节点赋予类别
node_types = []
for id_ in goods_id:
    node_types.append((id_, 'goods'))

for id_ in customer_id:
    node_types.append((id_+num_goods, 'customer'))
def log_norm(arr):
    arr = np.log10(arr + 0.1)
    return arr / arr.max()
# 所有节点特征，用户特征使用购买次数作为特征，商品特征为1
node_features = {'feature':np.concatenate([np.ones_like(goods_raw.values), customer_raw.values]).reshape(-1,1).astype("float32")}
# 所有的边列表
edge_list = edge_raw.drop_duplicates(subset=None, keep='first', inplace=False).sort_values(['goods_id'])
edge_list['customer_id'] += num_goods
edge_list2 = edge_list[['goods_id', 'customer_id']]
edge_list = edge_list.values
edge_list2 = edge_list2.values
# 添加自环
edge_list3 = np.repeat(np.expand_dims(goods_id, axis=1), repeats=2, axis=1)
edge_list4 = np.repeat(np.expand_dims(customer_id, axis=1), repeats=2, axis=1)
# 无向图，所以将正向和反向列表拼接
edge_list = np.concatenate([edge_list, edge_list2, edge_list3, edge_list4], axis=0)
# 定义边列表，类别名称是'buy'
edge_list = {'buy': edge_list.tolist()}

# 测试节点index，即所有的用户节点id
# 前0到num_goods-1个节点表示商品，从num_goods到num_nodes表示用户
test_index = np.concatenate([np.arange(num_goods, num_nodes)]).reshape(-1,1).astype("int")

# 训练节点label
train_label = train_data['labels'].values
# 训练节点index，即在trin_raw中的customer_id，数值上从num_goods到num_nodes中取样
customer_id_train = train_data.index + num_goods
train_index = np.concatenate([customer_id_train]).reshape(-1,1).astype("int")


In [19]:
!pip install pgl -i https://mirror.baidu.com/pypi/simple

import paddle.fluid as fluid
import paddle.fluid.layers as fl
import pgl
from pgl import heter_graph
from pgl import heter_graph_wrapper


## 异构图GCN模型

对每一种边分别全连接，实际上这里只用了一种边('buy')。


In [20]:
def HeteroGCNLayer(gw, edge_types, features, hidden_size=10, norm=None, name=''):
    def send_func(src_feat, dst_feat, edge_feat):
        if norm is not None:
            return src_feat['h'] * src_feat['norm']
        else:
            return src_feat['h'] 
    def recv_func(feat):
        return fluid.layers.sequence_pool(feat, pool_type='sum')

    assert len(edge_types) == len(features)
    output = []
    for i in range(len(edge_types)):
        msg = gw[edge_types[i]].send(send_func, nfeat_list=[('h', features[i]), ('norm', norm[i])])
        out = gw[edge_types[i]].recv(msg, recv_func)
        out = fluid.layers.fc(out, size=hidden_size, act='relu')
        if norm is not None:
            out = out * norm[i]
        output.append(out)
    # list of matrix
    return output

In [21]:
# 生产图数据
g = heter_graph.HeterGraph(num_nodes=num_nodes,
                            edges=edge_list,
                            node_types=node_types,
                            node_feat=node_features,)
                            # edge_feat=edges_weight)
# 根据节点的度提取归一化特征
indegree = g.indegree()
norm = np.zeros_like(indegree, dtype="float32")
norm[indegree > 0] = np.power(indegree[indegree > 0], -0.5)
g.node_feat["norm"] = np.expand_dims(norm, -1)

In [22]:
place = fluid.CUDAPlace(0) 
train_program = fluid.Program()
startup_program = fluid.Program()
hidden_size = 16
# 网络结构
with fluid.program_guard(train_program, startup_program):
# create a GraphWrapper as a container for graph data
    gw = heter_graph_wrapper.HeterGraphWrapper(name='heter_graph',
                                        place = place,
                                        edge_types = g.edge_types_info(),
                                        node_feat=g.node_feat_info(),
                                        edge_feat=g.edge_feat_info())

    edge_types = ['buy']
    features = []
    norms = []
    for edge_type in edge_types:
        features.append(gw[edge_type].node_feat['feature'])
        norms.append(gw[edge_type].node_feat["norm"])
    output = HeteroGCNLayer(gw, edge_types, features, hidden_size, norms)
    output1 = []
    for i in range(len(output)):
        output1.append(fluid.layers.dropout(
            output[i], 0.5, dropout_implementation='upscale_in_train'))
    output = output1
    # output = HeteroGCNLayer(gw, edge_types, output, hidden_size)

    node_index = fluid.layers.data("node_index", shape=[None, 1], dtype="int64", append_batch_size=False)
    output1 = []
    for i in range(len(output)):
        output1.append(fluid.layers.gather(output[i], node_index))
    output = output1
    output = fl.concat(input=output, axis=1)

    output = fluid.layers.fc(output, size=4, bias_attr=False, act='relu', name='fc1')
    logits = fluid.layers.fc(output, size=2, bias_attr=False, act=None, name='fc2')
    # pred = fluid.layers.softmax(logits, axis=1)
    node_label = fluid.layers.data("node_label", shape=[None, 1], dtype="float32")
    label_64 = fluid.layers.cast(node_label, 'int64')
    # loss = fluid.layers.cross_entropy(input=pred, label=fluid.layers.concat([1-node_label, node_label], axis=1), soft_label=True)
    # loss = fluid.layers.sigmoid_cross_entropy_with_logits(x=logits, label=node_label,normalize=False)
    loss, pred = fluid.layers.softmax_with_cross_entropy(logits=logits, label=label_64, soft_label=False, return_softmax=True, axis=1)
    loss = fluid.layers.mean(loss) 

    # def bce_loss(pred, label, epsilon=1e-05): # 标签都是 0或1，但是计算上log(0)不合法，所以一般将label和pred卡到[eps, 1-eps]范围内
    #     label = fluid.layers.clip(label, epsilon, 1-epsilon)
    #     pred = fluid.layers.clip(pred, epsilon, 1-epsilon) # 防止出现log(0)

    #     loss = -1 * (label * fluid.layers.log(pred) + (1 - label) * fluid.layers.log(1 - pred))
    #     loss = fluid.layers.reduce_mean(loss)
    #     return loss
    # loss = bce_loss(pred, node_label)
    # p2 = fluid.layers.concat([1-pred, pred], axis=1)
    acc = fluid.layers.accuracy(input=pred, label=fluid.layers.cast(node_label, 'int64'), k=1)
    output = pred[:,1]



test_program = train_program.clone(for_test=True)
with fluid.program_guard(train_program, startup_program): 
    adam = fluid.optimizer.Adam(
        learning_rate=1e-2,
        regularization=fluid.regularizer.L2DecayRegularizer(
            regularization_coeff=0.0005))
    adam.minimize(loss)

## 开始训练

In [ ]:
label = train_label.reshape(-1,1).astype("float32")
index = train_index.reshape(-1,1).astype("int")

exe = fluid.Executor(place)
exe.run(startup_program)
feed_dict = gw.to_feed(g)

for epoch in range(20):
    feed_dict['node_index'] = index
    feed_dict['node_label'] = label

    train_loss, acc_, pred_ = exe.run(train_program, feed=feed_dict, fetch_list=[loss,acc,output,], return_numpy=True)
    # train_loss, pred_  = exe.run(train_program, feed=feed_dict, fetch_list=[loss, pred ], return_numpy=True)
    print('Epoch %d | Loss: %f | Acc: %f '%(epoch, train_loss[0], acc_[0]))

## 在训练的模型基础下，对全部节点输出类别

In [ ]:
feed_dict['node_index'] = test_index.reshape(-1,1).astype("int")
feed_dict["node_label"] = np.ones_like(test_index).reshape(-1,1).astype("float32")
test_pred = exe.run(test_program,
                    feed=feed_dict,
                    fetch_list=[pred],
                    return_numpy=True)


In [ ]:
test_pred[0].min(),test_pred[0].max()

In [ ]:
sns.boxplot(test_pred[0])

## 后处理和输出预测结果submission.csv

In [ ]:
max_ = np.median(test_pred[0]) 
# 区间选择归一化，减少map函数的计算量
def f(x):
    if x < max_:
        return 0
    elif x >  max_:
        return 1
    # else:
    #     return x / max_ 


In [ ]:
submission_list['result'] = test_pred[0].reshape(-1).tolist()
submission_list['result'] = submission_list['result'].map(f)
subm = pd.DataFrame(submission_list['result'])
subm.to_csv('submission.csv')

## 特征提取参考

In [ ]:
# 对数据进行预处理
def prerpocess(raw, train='train'):
    st = time.time()
    # 性别：0未知，1男，2女
    data = pd.DataFrame(raw.groupby('customer_id')['customer_gender'].last().fillna(0))
    # 添加商品相关信息
    data[['goods_id_last', 'goods_status_last', 'goods_price_last', 'goods_has_discount_last', 'goods_list_time_last', 'goods_delist_time_last']] = raw.groupby('customer_id')['goods_id', 'goods_status', 'goods_price', 'goods_has_discount', 'goods_list_time', 'goods_delist_time'].last()
    # 添加订单相关信息
    data[['order_total_num_last', 'order_amount_last',
       'order_total_payment_last', 'order_total_discount_last', 'order_pay_time_last',
       'order_status_last', 'order_count_last', 'is_customer_rate_last',
       'order_detail_status_last', 'order_detail_goods_num_last', 'order_detail_amount_last',
       'order_detail_payment_last', 'order_detail_discount_last']] = raw.groupby('customer_id')['order_total_num', 'order_amount',
       'order_total_payment', 'order_total_discount', 'order_pay_time',
       'order_status', 'order_count', 'is_customer_rate',
       'order_detail_status', 'order_detail_goods_num', 'order_detail_amount',
       'order_detail_payment', 'order_detail_discount'].last()
    # 添加商品原始价格统计字段
    data[['good_price_std', 'good_price_mean', 'good_price_min', 'good_price_max']] = raw.groupby('customer_id')['goods_price'].agg({'good_price_std':'std', 'good_price_mean':'mean', 'good_price_min':'min', 'good_price_max':'max'})
    # 添加订单实付金额统计字段
    data[['order_detail_payment_std', 'order_detail_payment_mean', 'order_detail_payment_min', 'order_detail_payment_max']] = raw.groupby('customer_id')['order_detail_payment'].agg({'order_detail_payment_std':'std', 'order_detail_payment_mean':'mean', 'order_detail_payment_min':'min', 'order_detail_payment_max':'max'})
    # 用户购买的订单数量
    data['count'] = raw.groupby('customer_id')['order_id'].nunique()
    # 用户购买的商品数量
    data['goods_count'] = raw.groupby('customer_id')['order_total_num'].sum()
    # 用户所在省份
    data['customer_province'] = raw.groupby('customer_id')['customer_province'].last()
    # 用户所在城市
    data['customer_city'] = raw.groupby('customer_id')['customer_city'].last()
    # 用户是否评价 统计结果（平均，综合）
    data[['is_customer_rate_ratio','is_customer_rate_sum']] = raw.groupby('customer_id')['is_customer_rate'].agg({'is_customer_rate_ratio':np.mean,'is_customer_rate_sum':np.sum})
    # 用户购买的goods数量，一个订单商品，即order_detail_id（goods_id）
    data['order_detail_count'] = raw.groupby('customer_id')['customer_id'].count()
    # 商品折扣统计属性（sum, ave）
    data[['goods_has_discount_sum','goods_has_discount_ave']] = raw.groupby('customer_id')['goods_has_discount'].agg({'goods_has_discount_sum':np.sum,'goods_has_discount_ave':np.mean})
    # 订单实付金额 统计属性（sum, ave）
    data[['order_total_payment_sum','order_total_ave_pay']] = raw.groupby('customer_id')['order_total_payment'].agg({'order_total_payment_sum':np.sum,'order_total_ave_pay':np.mean})
    # 订单商品数量 统计属性（sum, ave）
    data[['order_total_num_sum', 'order_total_num_ave']] = raw.groupby('customer_id')['order_total_num'].agg({'order_total_num_sum':np.sum,'order_total_num_ave':np.mean})

    # 时间转换
    def time2multi(x):
        t=datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
        return pd.Series([t.month,t.day,t.weekday(),t.hour,t.minute,t.second])
    # 订单付款时间
    data[['order_pay_time_last_m','order_pay_time_last_d','order_pay_time_last_week','order_pay_time_last_h','order_pay_time_last_min','order_pay_time_last_s']]=data['order_pay_time_last'].apply(time2multi)
    #data[['order_pay_time_last_m','order_pay_time_last_d','order_pay_time_last_week','order_pay_time_last_h','order_pay_time_last_min','order_pay_time_last_s']] = raw.groupby('customer_id')['order_pay_time_last_m','order_pay_time_last_d','order_pay_time_last_week','order_pay_time_last_h','order_pay_time_last_min','order_pay_time_last_s'].last()
    # 起始时间是从2013-01-01开始
    t_str='2013-01-01 00:00:00'
    t=datetime.datetime.strptime(t_str, '%Y-%m-%d %H:%M:%S')
    # 商品最新上架时间diff （距离起始时间）
    data['goods_list_time_diff'] = data['goods_list_time_last'].map(lambda x:(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')-t).days/364)
    # 商品最新下架时间diff （距离起始时间）
    data['goods_delist_time_diff'] = data['goods_delist_time_last'].map(lambda x:(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')-t).days/364)
    # 商品展示时间
    data['goods_diff'] = data['goods_delist_time_diff'] - data['goods_list_time_diff']
    # 付款时间diff (距离起始时间)
    data['order_pay_time_last_diff'] = data['order_pay_time_last'].map(lambda x:(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')-t).days/364)
    ed = time.time()
    # 输出preprocess计算时间
    print(ed-st)
    
    return data

In [ ]:
# 8月之前的数据作为训练集
train_raw = raw[raw['order_pay_time'] < '2013-07-31 23:59:59']
train_raw = prerpocess(train_raw)
# 8月份的数据作为label_raw
label_raw = set(raw[raw['order_pay_time'] > '2013-07-31 23:59:59']['customer_id'].dropna())
# 如果该用户在8月份完成了购买 label=1, 否则为0
train_raw['labels']=train_raw.index.map(lambda x:int(x in label_raw))
test = prerpocess(raw)

In [ ]:
# 这些时间，之前已经根据时间提取了特征 原始的格式没法直接参与训练
train_data = train_raw.drop(['goods_list_time_last', 'goods_delist_time_last', 'order_pay_time_last'], axis=1)
# 暂时没有处理customer_province, customer_city 可以先去掉
train_data = train_data.drop(['customer_province', 'customer_city'], axis=1)
# 分类变量
catel = ['order_pay_time_last_h', 'order_pay_time_last_week', 'order_pay_time_last_m', 'order_detail_status_last', 'order_status_last', 'goods_status_last', 'goods_id_last', 'customer_gender']